# Data Preprocessing and Cleaning

Data cleaning is an essential step in the process of preparing data for further processing and serving into a large language model. Without data cleaning, the language model would be limited in its ability to interpret and process the data accurately. Data cleaning helps to ensure that the data is in a consistent format, free of any inconsistencies or errors that could lead to incorrect results. It also helps to reduce the amount of time needed to process the data, as it eliminates the need to manually check for any inconsistencies or errors. Additionally, data cleaning can help to improve the accuracy of the language model by removing any irrelevant or incorrect data. By taking the time to properly clean the data, the language model can more accurately interpret and process the data, leading to better results.

> 📍 Fill out the missing pieces in the source source to get everything working (indicated by `#FIXME`).

## Count tokens

> **Important note**
>
> *You do not need to manually tokenize strings before feeding texts into the model. This will be done automatically once you put instructions into the `prompt` parameter. However, you can use the `tiktoken` library to check how a string is tokenized and count the numbers of tokens to calculate the cost of an API call. Learn more [here](https://platform.openai.com/docs/introduction/tokens).*

Tokenizing text strings is an important step in natural language processing (NLP) as it helps models like GPT-3 understand the structure of a text string. Tokenizing breaks a text string into smaller pieces called tokens, which can then be analyzed and used by the model. By understanding the structure of a text string, models can better understand the meaning of the text. Additionally, tokenizing helps to determine the cost of an Azure OpenAI Service API call, as usage is priced by token. Furthermore, different models use different encodings, so it is important to tokenize text strings in the appropriate format.

`tiktoken` supports three encodings used by Azure OpenAI Service models:

| Encoding name | Azure OpenAI Service models |
| ------------- | -------------- |
| gpt2 (or r50k_base) | Most GPT-3 models |
| p50k_base | Code models, text-davinci-002, text-davinci-003 |
| cl100k_base | text-embedding-ada-002 |

Tokens in English typically range from one character to one word (e.g. "t" or "great"), though some languages may have tokens that are shorter or longer than one character or word. Spaces are usually placed at the beginning of words (e.g. " is" instead of "is " or "+"is"). You can use the Tokenizer to quickly check how a string is tokenized.

To show it briefly, we will use `tiktoken` to tokenize a text string and see how the output looks like.

In [2]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
  Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
Using cached tiktoken-0.8.0-cp312-cp312-win_amd64.whl (883 kB)
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [1]:
import tiktoken

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode("Hello world, this is fun!")

[15496, 995, 11, 428, 318, 1257, 0]

In [2]:

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode("Hello world, this is fun!")

[15496, 995, 11, 428, 318, 1257, 0]

Write a script that shows the string tokens from an input phrase.

In [11]:
#FIXME
palabra = "Holi que tal estas"

encoding = tiktoken.get_encoding("p50k_base")
encoding.encode(palabra)

[39, 11106, 8358, 3305, 1556, 292]

Let's write a function to count the number of tokens in a text string.

In [64]:
def get_num_tokens_from_string(string: str, encoding_name: str='p50k_base') -> int:
    encoding = tiktoken.get_encoding(encoding_name)
    encoded_tokens = encoding.encode(string)
    return len(encoded_tokens)


print(get_num_tokens_from_string("Hello World, this is fun!"))

encoding = tiktoken.get_encoding("p50k_base")
encoded_tokens = encoding.encode("Hello World, this is fun!")
print(encoded_tokens)

7
[15496, 2159, 11, 428, 318, 1257, 0]


## Clean data

Next we'll perform some light data cleaning by removing redundant whitespace and cleaning up the punctuation to prepare the data for tokenization. Again, tokenization is not required for the model to work, but it is a good practice to do so to ensure that the data is in a consistent format and that the model is able to process the data correctly. Also, it makes sure that the request is not too long for the model as the maximum number of tokens for davinci is 2048, e.g., equivalent to around 2-3 pages of text.

> **Best Practices**
> - **Replace newlines with a single space**: Unless you're embedding code, we suggest replacing newlines (\n) in your input with a single space, as we have observed inferior results when newlines are present.

In [ ]:
import re

def normalize_text(string, sep_token = " \n "):
    """Normalize text by removing unnecessary characters and altering the format of words."""
    # make text lowercase
    string = string.lower()
    
    string = re.sub(r'\s+',  ' ', string).strip()
    string = re.sub(r". ,","",string)
    # remove all instances of multiple spaces
    string = string.replace("..",".")
    string = string.replace(". .",".")
    string = string.replace("\n", "")
    string = string.strip()
    return string

Generate some data to test the cleaning function.

In [39]:
lorem = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor             \n incididunt ut labore et dolore magna aliqua.\n " \
        "Ut enim ad minim veniam, \n quis nostrud          exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.\n " \
        "Duis aute irure dolor in reprehenderit in VOLUPTATE \n velit esse cillum dolore           eu fugiat nulla pariatur.\n" \
        "Excepteur sint occaecat cupidatat non proident, \n       sunt in culpa qui officia deserunt mollit anim id est laborum."

normalize_text(lorem) #FIXME

'lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'

## Ejercicio 1: Validar longitud de texto antes de enviar a OpenAI

Escribe una función llamada validate_text_length que valide si un texto supera un límite de tokens especificado. Si el texto supera el límite, la función debe dividirlo en segmentos de tamaño adecuado para enviarlos al modelo de OpenAI. Usa la codificación cl100k_base.



In [63]:
import tiktoken

def validate_text_length(text, max_tokens=2048, encoding_name='cl100k_base'):
    encoding = tiktoken.get_encoding(encoding_name)
    encoded_tokens = encoding.encode(text)
    
    segments = []
    start = 0
    while start < len(encoded_tokens):
        end = start + max_tokens
        segment = encoded_tokens[start:end]
        segments.append(encoding.decode(segment))
        start = end

    return segments

# Prueba la función
text = "Lorem ipsum " * 8000  # Texto largo
segments = validate_text_length(text)

# Verifica el número de segmentos y su longitud
print(f"Numero de segmentos: ", len(segments))  # Resultado esperado: Más de 1 segmento
print(f"Numero de tokens en el segmento: ", len(tiktoken.get_encoding('cl100k_base').encode(segments[0])))  # <= 2048 tokens
print(tiktoken.get_encoding('cl100k_base').encode(segments[0]))
##Resultado esperado:

##El texto se divide en segmentos adecuados para ser procesados por OpenAI.
##Cada segmento tendrá un número de tokens menor o igual al límite especificado.


Numero de segmentos:  8
Numero de tokens en el segmento:  2048
[33883, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439, 48031, 27439

## Ejercicio 2: Filtrar palabras prohibidas en un texto

Escribe una función llamada filter_prohibited_words que remueva palabras prohibidas de un texto antes de enviarlo a OpenAI. 

Como ejemplo queremos Eliminar palabras como: ['password', 'confidential', 'secret']. 



In [94]:
import re

def filter_prohibited_words(text, prohibited_words):
    #FIXME
    texto = text.split()
    for i, p in enumerate(texto):
        if p in prohibited_words:
            texto[i] = texto[i].replace(p, "[REDACTED]")
    return " ".join(texto)

# Prueba la función
text = "This document contains confidential and secret information."
prohibited_words = ['password', 'confidential', 'secret']

cleaned_text = filter_prohibited_words(text, prohibited_words) #FIXME
print(cleaned_text)  # Resultado esperado: "This document contains [REDACTED] and [REDACTED] information."
#Resultado esperado:
#El texto tendrá palabras prohibidas reemplazadas por [REDACTED].



This document contains [REDACTED] and [REDACTED] information.


## Ejercicio 3: Identificar idioma del texto

Crea una función detect_language que use langdetect para identificar el idioma de un texto y devuelva el código del idioma (e.g., en, es). Verifica el idioma antes de enviarlo al modelo OpenAI. Debes instalar la libreria langdetect


In [96]:
pip install langdetect

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------------------------------------- 10.2/981.5 kB ? eta -:--:--
     ---- --------------------------------- 112.6/981.5 kB 1.6 MB/s eta 0:00:01
     ------------------------- ------------ 655.4/981.5 kB 5.9 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993255 sha256=e5a90566fe49d2198de9dd5d4d5d52caf64a0114f95c1836b682791d8152f135
  Stored in directory: c:\users\alumno_ai\appdata\local\pip\cache\wheels\c1\67\88\e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [102]:
from langdetect import detect

def detect_language(text):
    """Detecta el idioma de un texto."""
    #FIXME
    return detect(text)

# Prueba la función
text = "Hola, este es un texto en español."
text2 = "Hello, this text is in english"
language = detect_language(text) #FIXME
language2 = detect_language(text2)
print(language)  # Resultado esperado: 'es' (para español)
print(language2)
##Resultado esperado:
##El idioma del texto detectado (es, en, etc.) será devuelto.


es
en


## Ejercicio 5: Generar prompts efectivos

Escribe una función generate_effective_prompt que genere un prompt optimizado para OpenAI, dado un tema y un objetivo. Usa plantillas como:
“Explain [tema] in simple terms for a [audiencia].”



In [107]:
def generate_effective_prompt(topic, audience="general audience"):
    """Genera un prompt optimizado para OpenAI."""
    #FIXME
    prompt = f"Explain {topic} in simple terms for a {audience}."
    return prompt

# Prueba la función
topic = "machine learning"
prompt = generate_effective_prompt(topic)  #FIXME 
print(prompt)  # Resultado esperado: "Explain machine learning in simple terms for a beginner."


Explain machine learning in simple terms for a general audience.


## Ejercicio 6: Medir costo estimado del uso de OpenAI

Crea una función estimate_cost que calcule el costo estimado de una solicitud a OpenAI basado en el número de tokens. Supón un costo de $0.02 por 1000 tokens.



In [145]:
def estimate_cost(text, encoding_name='cl100k_base', cost_per_1k_tokens=0.02):
    """Calcula el costo estimado de un texto basado en el número de tokens."""
    #FIXME
    encoding = tiktoken.get_encoding(encoding_name)
    encoded_tokens = encoding.encode(text)
    cost1k = len(encoded_tokens) / 1000
    cost = cost1k * cost_per_1k_tokens
    return cost


# Prueba la función
text = "This is a sample text." * 1000
cost = estimate_cost(text)
print(cost)  # Resultado esperado: Costo aproximado basado en la cantidad de tokens



0.10002000000000001


## Ejercicio 7: Limpiar texto JSON para OpenAI

Escribe una función clean_json_text que tome un JSON como entrada y limpie todas las claves y valores de caracteres no ASCII.


In [146]:
import json
import re

def clean_value(value):
    """Elimina caracteres no ASCII de un valor."""
    if isinstance(value, str):
        return re.sub(r'[^\x00-\x7F]+', '', value)
    return value

def clean_json_text(json_obj):
    """Limpia texto JSON eliminando caracteres no ASCII."""
    return {k: clean_value(v) for k, v in json_obj.items()}

# Prueba la función
data = {"key1": "Hello 😊", "key2": "Café and thé"}
cleaned_data = clean_json_text(data)
print(cleaned_data)  # Resultado esperado: {"key1": "Hello ", "key2": "Caf and th"}
##Resultado esperado:
##Un JSON sin caracteres no ASCII.

{'key1': 'Hello ', 'key2': 'Caf and th'}
